## PGX - Phuture GameFi Index - Methodology Template

## Configuring packages

In [1]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import MethodologyBase
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [2]:
min_mcap =  0.25e8
min_weight = 0.005
max_weight = 0.3
circ_supply_threshold = 0.1
liveness_threshold = 1 # In days
liquidity_consistency = 90 # In days
coingecko_category = ["gaming"]

# Calculating max slippage based on liquidity threshold
liq = 5*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')

methodology_class = MethodologyBase(
                                'arbitrum-one',
                                min_mcap,
                                min_weight,
                                max_weight,
                                65535,
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                -0.12,
                                100000,
                                coingecko_category)

## Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data spanning at least 6 months prior to the date of inclusion in the index.
    1. The project should have an active community.
    1. No rebasing or deflationary tokens.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project must have a minimum circulating market cap of $50mm.
   <br>
1. **Pricing requirmeents**
    1. Token should have a reliable price feed from one of our authorised providers. 
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 20% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The price should experience no more than 12% price impact when executing a $100k trade.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.
1. **Weighting requirements**
    1. The maximum weight any one token can have is 30%.
    1. All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
    1. Any asset with a weight below 0.5% will be removed from the index.


## Run Methodology

In [3]:

results,liquidity = methodology_class.main(
    df_to_remove=[stablecoins,ctokens,atokens],
    add_category_assets=['decentraland','gam3s-gg'],
    remove_category_assets=['floki','bluzelle','xai-blockchain  '],
    enable_liquidity_score=True
    )
liquidity

Manually calculating slippage for immutable-x on ethereum
Manually calculating slippage for gala on ethereum
Manually calculating slippage for axie-infinity on ethereum
ronin not supported
Manually calculating slippage for axie-infinity on binance-smart-chain
harmony-shard-0 not supported
energi not supported
Manually calculating slippage for beam-2 on ethereum
Manually calculating slippage for beam-2 on binance-smart-chain
Manually calculating slippage for the-sandbox on ethereum
harmony-shard-0 not supported
Manually calculating slippage for the-sandbox on polygon-pos
energi not supported
Manually calculating slippage for decentraland on ethereum
Manually calculating slippage for decentraland on polygon-pos
Manually calculating slippage for apecoin on ethereum
Manually calculating slippage for apecoin on polygon-pos
Manually calculating slippage for illuvium on ethereum
energi not supported
Manually calculating slippage for echelon-prime on ethereum
Manually calculating slippage for 

,arbitrum-one,avalanche,base,binance-smart-chain,ethereum,polygon-pos,optimal chain,best slippage,optimal slippage,best slippage chain,liquidity score
id,,,,,,,,,,,
illuvium,NaN,NaN,NaN,NaN,-0.004919,NaN,ethereum,-0.004919,-0.004919,ethereum,100.000000
echelon-prime,NaN,NaN,-0.178803,NaN,-0.008637,NaN,ethereum,-0.008637,-0.008637,ethereum,99.629955
beam-2,NaN,NaN,NaN,-0.260079,-0.016783,NaN,ethereum,-0.016783,-0.016783,ethereum,98.819312
mobox,NaN,NaN,NaN,-0.022515,NaN,NaN,binance-smart-chain,-0.022515,-0.022515,binance-smart-chain,98.248959
magic,-0.037786,NaN,NaN,NaN,-0.785291,NaN,arbitrum-one,-0.037786,-0.037786,arbitrum-one,96.729272
wilder-world,NaN,NaN,NaN,NaN,-0.039909,NaN,ethereum,-0.039909,-0.039909,ethereum,96.517950
dream-machine-token,-0.657477,NaN,NaN,NaN,-0.045572,NaN,ethereum,-0.045572,-0.045572,ethereum,95.954439
heroes-of-mavia,NaN,NaN,-0.050340,NaN,-0.092534,NaN,base,-0.050340,-0.050340,base,95.479945
immutable-x,NaN,NaN,NaN,NaN,-0.050418,NaN,ethereum,-0.050418,-0.050418,ethereum,95.472174


## Show Results

In [4]:
results


,symbol,market_cap,weight,weight_converted,address,blockchain
id,,,,,,
immutable-x,IMX,1.892274e+09,0.300000,19661,0xf57e7e7c23978c3caec3c3548e3d615c346e79ff,ethereum
beam-2,BEAM,7.284587e+08,0.300000,19661,0x62d0a8458ed7719fdaf978fe5929c6d342b0bfce,ethereum
illuvium,ILV,3.875814e+08,0.182600,11967,0x767fe9edc9e0df98e07454847909b5e959d7ca0e,ethereum
magic,MAGIC,1.232331e+08,0.056159,3681,0x539bde0d7dbd336b79148aa742883198bbf60342,arbitrum-one
wilder-world,WILD,9.394471e+07,0.042719,2799,0x2a3bff78b79a009976eea096a51a948a3dc00e34,ethereum
xai-blockchain,XAI,9.856308e+07,0.042294,2771,0x4cb9a7ae498cedcbb5eae9f25736ae7d428c9d66,arbitrum-one
mobox,MBOX,5.660446e+07,0.026201,1717,0x3203c9e46ca618c8c1ce5dc67e7e9d75f5da2377,binance-smart-chain
radio-caca,RACA,5.645483e+07,0.025361,1662,0x12bb890508c125661e03b09ec06e404bc9289040,binance-smart-chain
aavegotchi,GHST,5.514068e+07,0.024667,1616,0x385eeac5cb85a38a9a07a70c73e0a3271cfb54a7,polygon-pos


In [5]:
liquidity.head(30)

,arbitrum-one,avalanche,base,binance-smart-chain,ethereum,polygon-pos,optimal chain,best slippage,optimal slippage,best slippage chain,liquidity score
id,,,,,,,,,,,
illuvium,NaN,NaN,NaN,NaN,-0.004919,NaN,ethereum,-0.004919,-0.004919,ethereum,100.000000
echelon-prime,NaN,NaN,-0.178803,NaN,-0.008637,NaN,ethereum,-0.008637,-0.008637,ethereum,99.629955
beam-2,NaN,NaN,NaN,-0.260079,-0.016783,NaN,ethereum,-0.016783,-0.016783,ethereum,98.819312
mobox,NaN,NaN,NaN,-0.022515,NaN,NaN,binance-smart-chain,-0.022515,-0.022515,binance-smart-chain,98.248959
magic,-0.037786,NaN,NaN,NaN,-0.785291,NaN,arbitrum-one,-0.037786,-0.037786,arbitrum-one,96.729272
wilder-world,NaN,NaN,NaN,NaN,-0.039909,NaN,ethereum,-0.039909,-0.039909,ethereum,96.517950
dream-machine-token,-0.657477,NaN,NaN,NaN,-0.045572,NaN,ethereum,-0.045572,-0.045572,ethereum,95.954439
heroes-of-mavia,NaN,NaN,-0.050340,NaN,-0.092534,NaN,base,-0.050340,-0.050340,base,95.479945
immutable-x,NaN,NaN,NaN,NaN,-0.050418,NaN,ethereum,-0.050418,-0.050418,ethereum,95.472174


In [6]:
##results.to_excel("g3 gaming index.xlsx")